In [ ]:
# install py-spark
!pip install --quiet pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 56.5 MB/s 


In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Start Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("OLAP-project") \
    .getOrCreate()

In [ ]:
# A function to create table from a CSV
def csvToTable(file_name, table_name):
  db_name = ""
  #table_name = db_name + table_name
  df = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep", ",") \
  .csv( "gdrive/My Drive/Big data/project" + file_name )
  df.createOrReplaceTempView( table_name +"df" ) 

  sql_str = "drop table IF EXISTS " + table_name
  spark.sql( sql_str )

  sql_str = "create table " + table_name \
  + " USING PARQUET AS" \
  + " SELECT * FROM " + table_name + "df"
  spark.sql( sql_str )

  sql_str = "describe " + table_name
  print("Table " + table_name + " Done Successfully!")
  spark.sql( sql_str ).show()

  spark.catalog.dropTempView( table_name + "df" )

In [ ]:
  # Create Tables
csvToTable("/appearances.csv", "appearances")
csvToTable("/players.csv", "players")
csvToTable("/shots.csv", "shots")
csvToTable("/games.csv", "games")


Table appearances Done Successfully!
+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|       gameID|      int|   null|
|     playerID|      int|   null|
|        goals|      int|   null|
|     ownGoals|      int|   null|
|        shots|      int|   null|
|       xGoals|   double|   null|
|  xGoalsChain|   double|   null|
|xGoalsBuildup|   double|   null|
|      assists|      int|   null|
|    keyPasses|      int|   null|
|     xAssists|   double|   null|
|     position|   string|   null|
|positionOrder|      int|   null|
|   yellowCard|      int|   null|
|      redCard|      int|   null|
|         time|      int|   null|
| substituteIn|      int|   null|
|substituteOut|      int|   null|
|     leagueID|      int|   null|
+-------------+---------+-------+

Table players Done Successfully!
+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|playerID|      int|   null|
|    name|   string|   null|
+-

In [ ]:
#Viewing appearances table
sql_str = "select * from appearances limit 10"
spark.sql( sql_str ).show()

+------+--------+-----+--------+-----+-----------------+------------------+------------------+-------+---------+------------------+--------+-------------+----------+-------+----+------------+-------------+--------+
|gameID|playerID|goals|ownGoals|shots|           xGoals|       xGoalsChain|     xGoalsBuildup|assists|keyPasses|          xAssists|position|positionOrder|yellowCard|redCard|time|substituteIn|substituteOut|leagueID|
+------+--------+-----+--------+-----+-----------------+------------------+------------------+-------+---------+------------------+--------+-------------+----------+-------+----+------------+-------------+--------+
|    81|     560|    0|       0|    0|              0.0|               0.0|               0.0|      0|        0|               0.0|      GK|            1|         0|      0|  90|           0|            0|       1|
|    81|     557|    0|       0|    0|              0.0| 0.106513060629368| 0.106513060629368|      0|        1| 0.106513060629368|      DR|

In [ ]:
#Viewing players table
sql_str = "select * from players limit 10"
spark.sql( sql_str ).show()

+--------+-------------------+
|playerID|               name|
+--------+-------------------+
|     560|      Sergio Romero|
|     557|     Matteo Darmian|
|     548|        Daley Blind|
|     628|     Chris Smalling|
|    1006|          Luke Shaw|
|     551|Morgan Schneiderlin|
|     654|    Michael Carrick|
|     554|          Juan Mata|
|     555|      Memphis Depay|
|     631|       Ashley Young|
+--------+-------------------+



In [ ]:
#Viewing games table
sql_str = "select * from shots limit 10"
spark.sql( sql_str ).show()

+------+---------+----------+------+--------------+------------+---------+-----------+------------------+-----------------+-----------------+
|gameID|shooterID|assisterID|minute|     situation|  lastAction| shotType| shotResult|             xGoal|        positionX|        positionY|
+------+---------+----------+------+--------------+------------+---------+-----------+------------------+-----------------+-----------------+
|    81|      554|        NA|    27|DirectFreekick|    Standard| LeftFoot|BlockedShot| 0.104346722364426|0.794000015258789|0.420999984741211|
|    81|      555|       631|    27|      SetPiece|        Pass|RightFoot|BlockedShot| 0.064342200756073|             0.86|0.627000007629395|
|    81|      554|       629|    35|      OpenPlay|        Pass| LeftFoot|BlockedShot|0.0571568161249161|0.843000030517578|0.332999992370605|
|    81|      554|        NA|    35|      OpenPlay|      Tackle| LeftFoot|MissedShots|0.0921413898468018|0.848000030517578|0.532999992370605|
|    8

In [ ]:
#Viewing games table
sql_str = "select * from games limit 10"
spark.sql( sql_str ).show()

+------+--------+------+-------------------+----------+----------+---------+---------+---------------+---------------+---------------+-----------------+-----------------+-----+-----+-----+----+----+----+----+---+----+----+----+-----+----+---+----+----+----+----+----+----+-----+
|gameID|leagueID|season|               date|homeTeamID|awayTeamID|homeGoals|awayGoals|homeProbability|drawProbability|awayProbability|homeGoalsHalfTime|awayGoalsHalfTime|B365H|B365D|B365A| BWH| BWD| BWA| IWH|IWD| IWA| PSH| PSD|  PSA| WHH|WHD| WHA| VCH| VCD| VCA|PSCH|PSCD| PSCA|
+------+--------+------+-------------------+----------+----------+---------+---------+---------------+---------------+---------------+-----------------+-----------------+-----+-----+-----+----+----+----+----+---+----+----+----+-----+----+---+----+----+----+----+----+----+-----+
|    81|       1|  2015|2015-08-08 15:45:00|        89|        82|        1|        0|         0.2843|         0.3999|         0.3158|                1|           

# CUBE BUILDING

#appearance cube

##1.Get player id name goals position 

In [ ]:

sql_str = "SELECT a.playerId,p.name, SUM(goals)as goals, position FROM appearances a  " \
  + " JOIN players p ON( a.playerId = p.playerId)"\
  + "GROUP BY p.name, a.playerID,position WITH CUBE"
appearances_cube = spark.sql( sql_str )
appearances_cube.show()
appearances_cube.write.mode("overwrite").csv('appearances_cube3d.csv')

+--------+------------------+-----+--------+
|playerID|              name|goals|position|
+--------+------------------+-----+--------+
|     927|  Sebasti�n Coates|    0|      DC|
|     863|  Jonathan Walters|    6|     AMR|
|     601|       Arouna Kon�|    3|     AML|
|     815|    Alex Pritchard|    0|     Sub|
|     897|            Juanmi|   12|     Sub|
|     823|Vadis Odjidja-Ofoe|    0|     Sub|
|     939|      Mauro Z�rate|    0|      MR|
|    1764|     Ramadan Sobhi|    2|     Sub|
|    1954|  Samuel Souprayen|    0|      DL|
|    1888| Antonio Di Natale|   14|      FW|
|    1953|      Karim Laribi|    0|     Sub|
|    1279|  Roberto Vitiello|    0|     Sub|
|    1442|     Saphir Ta�der|    0|     DMC|
|    1815|              Alex|    0|     Sub|
|    1859|    Matteo Ciofani|    0|      DR|
|    1864|        Mirko Gori|    0|     Sub|
|    1480|   Gianluca Pegolo|    0|      GK|
|    1899|  Lorenzo Pasciuti|    0|      ML|
|    1899|  Lorenzo Pasciuti|    2|      MR|
|    1446|

In [ ]:
appearances_cube.createOrReplaceTempView( "appearances_cube" )

#shots cube

##2. Get shooter id, situation, minute, shot result 

In [ ]:
sql_str = "SELECT shooterId, situation, minute , shotResult FROM shots group by shooterId, situation, shotResult,minute WITH CUBE ORDER BY shooterId"
shots_cube = spark.sql( sql_str )
shots_cube.show()
shots_cube.write.mode("overwrite").csv('shots_cube3d.csv')

+---------+--------------+------+-----------+
|shooterId|     situation|minute| shotResult|
+---------+--------------+------+-----------+
|     null|      OpenPlay|    82| ShotOnPost|
|     null|      SetPiece|    62|BlockedShot|
|     null|    FromCorner|    62|       Goal|
|     null|      SetPiece|    14|MissedShots|
|     null|      OpenPlay|    12|       null|
|     null|          null|  null|BlockedShot|
|     null|    FromCorner|     2|       Goal|
|     null|    FromCorner|     9| ShotOnPost|
|     null|DirectFreekick|    80| ShotOnPost|
|     null|       Penalty|    59|  SavedShot|
|     null|      OpenPlay|    75|       Goal|
|     null|      SetPiece|    68|MissedShots|
|     null|          null|    57|       null|
|     null|      OpenPlay|    15|       null|
|     null|      OpenPlay|     9|       null|
|     null|          null|     3|  SavedShot|
|     null|      OpenPlay|    76|MissedShots|
|     null|    FromCorner|    44|       null|
|     null|          null|    58| 

In [ ]:
appearances_cube.createOrReplaceTempView( "shots_cube" )

#games cube


##3. get game_id, season, homeGoals, awayGoals from games table

In [ ]:
sql_str = "SELECT gameId, season, SUM(homeGoals) as home_goals , SUM(awayGoals) as away_goals FROM games"\
+" group by gameId, season  WITH CUBE ORDER BY gameId"
games_cube = spark.sql( sql_str )
games_cube.show()
games_cube.write.mode("overwrite").csv('games_cube3d.csv')

+------+------+----------+----------+
|gameId|season|home_goals|away_goals|
+------+------+----------+----------+
|  null|  2016|      2967|      2206|
|  null|  2017|      2794|      2153|
|  null|  2014|      2736|      2062|
|  null|  2020|      2696|      2418|
|  null|  2018|      2819|      2200|
|  null|  2019|      2663|      2153|
|  null|  null|     19441|     15300|
|  null|  2015|      2766|      2108|
|    81|  null|         1|         0|
|    81|  2015|         1|         0|
|    82|  null|         0|         1|
|    82|  2015|         0|         1|
|    83|  null|         2|         2|
|    83|  2015|         2|         2|
|    84|  null|         4|         2|
|    84|  2015|         4|         2|
|    85|  2015|         1|         3|
|    85|  null|         1|         3|
|    86|  null|         2|         2|
|    86|  2015|         2|         2|
+------+------+----------+----------+
only showing top 20 rows



In [ ]:
games_cube.createOrReplaceTempView( "games_cube" )

#slicing

##4.Total goals and assists of FW players 

In [ ]:
sql_str = "SELECT * from appearances_cube where position = 'FW' "
spark.sql( sql_str ).show()

+--------+------------------+-----+--------+
|playerID|              name|goals|position|
+--------+------------------+-----+--------+
|    1888| Antonio Di Natale|   14|      FW|
|    1473|   Matteo Politano|    8|      FW|
|    6036|Rajiv van La Parra|    0|      FW|
|    5169| Youssef En-Nesyri|   32|      FW|
|    1577|       Marko Pjaca|    2|      FW|
|     594|              null|  123|      FW|
|     940|              null|    7|      FW|
|    null|       Libor Koz�k|    0|      FW|
|    null|         Marco Sau|   12|      FW|
|    null|       Timo Werner|   74|      FW|
|     276|              null|    4|      FW|
|    null|      Sergi Enrich|   28|      FW|
|    null|         Casades�s|    8|      FW|
|    3683|              null|    1|      FW|
|    null|   Julian Schieber|    6|      FW|
|    null|     Rub�n Sobrino|    6|      FW|
|    null|  Prejuce Nakoulma|    8|      FW|
|    null|     George Puscas|    1|      FW|
|    4366|              null|    0|      FW|
|    4718|

##5.Highest goal ratio playerid and his goals in season 2020.

In [ ]:

sql_str = "SELECT shooterId, p.name, SUM(a.goals), MAX(xGoal) from shots"\
 +" s JOIN players p ON(s.shooterId = p.playerId) JOIN games g ON(s.gameId = g.gameId)"\
 +" JOIN appearances a ON(s.shooterId = a.playerId)"\
 +" where g.season = '2020' GROUP BY s.shooterId, p.name"\
 +" ORDER BY s.shooterId"

spark.sql( sql_str ).show()

+---------+--------------------+----------+------------------+
|shooterId|                name|sum(goals)|        max(xGoal)|
+---------+--------------------+----------+------------------+
|        3|      Luca Caldirola|        21| 0.130910724401474|
|       22|         John Brooks|       180|  0.51889580488205|
|       23| Marvin Plattenhardt|        18|0.0501803532242775|
|       27|     Vladimir Darida|       459| 0.474260807037354|
|       28|     Mitchell Weiser|        18|0.0583156235516071|
|       29|      Vedad Ibisevic|        90| 0.646249175071716|
|       38|    Daniel Brosinski|       100|  0.75777667760849|
|       40|         Stefan Bell|        88|0.0900668501853943|
|       42|Julian Baumgartli...|        49| 0.689103305339813|
|       43|         Danny Latza|       132| 0.605999708175659|
|       47|      Yoshinori Muto|       484| 0.675653636455536|
|       48|        Jhon C�rdoba|      1632| 0.583697021007538|
|       52|      Alexander Hack|        21|0.0974528416

##6.Gameid, awayGoals, homeProbability, awayProbability where a home team lost by 4 or more goals.

In [ ]:
sql_str = "SELECT Gameid,awayGoals, homeProbability, awayProbability from games"\
" where awayGoals >=4 "
spark.sql( sql_str ).show()

+------+---------+---------------+---------------+
|Gameid|awayGoals|homeProbability|awayProbability|
+------+---------+---------------+---------------+
|   106|        4|         0.1041|         0.7202|
|   144|        5|         0.0518|         0.8594|
|   178|        5|         0.0068|          0.961|
|   208|        4|         0.0916|         0.7435|
|   265|        4|         0.5398|         0.2041|
|   286|        4|         0.0471|         0.7914|
|   301|        5|         0.2233|         0.4551|
|   339|        6|         0.0134|         0.8843|
|   384|        4|         0.0442|         0.8396|
|   386|        4|         0.4982|         0.2203|
|   414|        4|         0.0203|         0.9004|
|   421|        4|         0.0697|         0.8242|
|   422|        4|         0.0702|         0.7652|
|   439|        4|         0.3543|         0.3887|
|   469|        4|         0.2393|         0.4748|
|   472|        4|         0.1427|         0.6019|
|   493|        4|         0.07

# Dicing

##7.Get the player name, situation and last action when shot result is goal and minute is 90

In [ ]:
sql_str = "SELECT s.shooterId, p.name, situation, lastAction  FROM shots s JOIN players p ON (s.shooterId = p.playerId)"\
 + "WHERE shotResult='Goal' and minute = 90 " \
  + "ORDER BY s.shooterId"
spark.sql( sql_str ).show()

+---------+--------------------+--------------+-----------+
|shooterId|                name|     situation| lastAction|
+---------+--------------------+--------------+-----------+
|       26|       Salomon Kalou|       Penalty|   Standard|
|       26|       Salomon Kalou|    FromCorner|   HeadPass|
|       26|       Salomon Kalou|       Penalty|   Standard|
|       29|      Vedad Ibisevic|      OpenPlay|       Pass|
|       47|      Yoshinori Muto|    FromCorner|      Cross|
|       48|        Jhon C�rdoba|      OpenPlay|     Aerial|
|       58|   Georg Niedermeier|    FromCorner|       Pass|
|       62|          Lukas Rupp|    FromCorner|       None|
|       65|         Timo Werner|      OpenPlay|       Pass|
|       81|     Jonathan Schmid|      OpenPlay|   HeadPass|
|       83|       Kevin Volland|      OpenPlay|    Rebound|
|       88|            Mark Uth|      OpenPlay|       Pass|
|      104|       Mevl�t Erdinc|      OpenPlay|    Rebound|
|      110|Theodor Gebre Sel...|    From

##8.Getting the values of shot result at minute 60 from 2019 season

In [ ]:
sql_str = "SELECT g.gameID, season, s.minute, s.shotResult from games_cube AS g JOIN shots AS s ON(g.gameId = s.gameID)"\
 +"where g.season = '2019' AND s.minute = 60 "
spark.sql( sql_str ).show()

+------+------+------+-----------+
|gameID|season|minute| shotResult|
+------+------+------+-----------+
| 11645|  2019|    60|MissedShots|
| 11653|  2019|    60|MissedShots|
| 11655|  2019|    60|       Goal|
| 11657|  2019|    60|  SavedShot|
| 11657|  2019|    60|BlockedShot|
| 11659|  2019|    60|  SavedShot|
| 11659|  2019|    60| ShotOnPost|
| 11661|  2019|    60|MissedShots|
| 11662|  2019|    60|BlockedShot|
| 11664|  2019|    60|  SavedShot|
| 11666|  2019|    60|BlockedShot|
| 11670|  2019|    60|BlockedShot|
| 11672|  2019|    60|  SavedShot|
| 11677|  2019|    60|BlockedShot|
| 11680|  2019|    60|  SavedShot|
| 11681|  2019|    60|MissedShots|
| 11683|  2019|    60|  SavedShot|
| 11684|  2019|    60|MissedShots|
| 11686|  2019|    60|  SavedShot|
| 11688|  2019|    60|BlockedShot|
+------+------+------+-----------+
only showing top 20 rows



#9.No of shots of players made where result = goal minute =90 and shots > 5.

In [ ]:
sql_str = "SELECT shooterId , p.name, SUM(a.shots) as shots , shotResult, minute  FROM shots s " \
  + "JOIN appearances a ON(s.shooterId = a.playerID)"\
  +" JOIN players p ON(s.shooterId = p.playerID) WHERE a.shots > 5 AND shotResult = 'Goal' AND minute = 90"\
  +" GROUP BY shooterId, p.name, shotResult, minute"
spark.sql( sql_str ).show()

+---------+--------------------+-----+----------+------+
|shooterId|                name|shots|shotResult|minute|
+---------+--------------------+-----+----------+------+
|      337|          Leroy San�|   18|      Goal|    90|
|      942|     Florian Thauvin|  206|      Goal|    90|
|     1518|     Rodrigo Palacio|   13|      Goal|    90|
|      717|     Baf�timbi Gomis|   38|      Goal|    90|
|     3577|      Cheick Diabat�|    7|      Goal|    90|
|      360|         �d�m Szalai|   21|      Goal|    90|
|     1723|     Roberto Pereyra|    6|      Goal|    90|
|     1728|   Fernando Llorente|   24|      Goal|    90|
|     4119|       Alberto Bueno|   40|      Goal|    90|
|     5222|     Joel Pohjanpalo|   13|      Goal|    90|
|     3544|        Jeremie Boga|    6|      Goal|    90|
|      888|     Xherdan Shaqiri|    6|      Goal|    90|
|     1291|            Hernanes|   12|      Goal|    90|
|     3570|  Benjamin Moukandjo|   12|      Goal|    90|
|      802|         Diego Costa

##10.Gameid, homegoals, awaygoals and awayprobability  where awayprobability was greater than homeprobability and awayteam goals where 3 more than home team goals

In [ ]:
sql_str = "SELECT gameID, homeGoals, awayGoals, awayProbability from games"\
" where awayProbability > homeProbability AND awayGoals > 3*homeGoals "
spark.sql( sql_str ).show()

+------+---------+---------+---------------+
|gameID|homeGoals|awayGoals|awayProbability|
+------+---------+---------+---------------+
|    90|        0|        3|         0.7687|
|    91|        0|        1|         0.4291|
|    92|        0|        3|         0.4619|
|   108|        0|        2|         0.6532|
|   111|        0|        1|         0.9348|
|   116|        0|        3|          0.467|
|   123|        0|        1|         0.9174|
|   128|        0|        1|         0.4623|
|   149|        0|        1|         0.7996|
|   152|        0|        1|         0.3847|
|   166|        0|        3|         0.5927|
|   168|        0|        3|         0.9274|
|   170|        0|        1|         0.7365|
|   174|        0|        1|         0.4872|
|   175|        0|        2|         0.3846|
|   178|        1|        5|          0.961|
|   185|        0|        3|         0.8712|
|   196|        0|        1|         0.8173|
|   203|        0|        1|         0.4862|
|   205|  

# ROLL UP and DRILL DOWN

##11. Get id, season, homeGoals, homeProbability from games table with roll up


In [ ]:
sql_str = "SELECT gameId, season, homeGoals , homeProbability FROM games"\
+" group by gameId, season, homeGoals, homeProbability WITH ROLLUP ORDER BY gameId "
shots_roll_up = spark.sql( sql_str )
shots_roll_up.show()

+------+------+---------+---------------+
|gameId|season|homeGoals|homeProbability|
+------+------+---------+---------------+
|  null|  null|     null|           null|
|    81|  2015|     null|           null|
|    81|  null|     null|           null|
|    81|  2015|        1|         0.2843|
|    81|  2015|        1|           null|
|    82|  2015|     null|           null|
|    82|  2015|        0|         0.3574|
|    82|  null|     null|           null|
|    82|  2015|        0|           null|
|    83|  2015|     null|           null|
|    83|  null|     null|           null|
|    83|  2015|        2|           null|
|    83|  2015|        2|         0.2988|
|    84|  2015|        4|           null|
|    84|  2015|        4|         0.6422|
|    84|  2015|     null|           null|
|    84|  null|     null|           null|
|    85|  null|     null|           null|
|    85|  2015|        1|           null|
|    85|  2015|        1|         0.1461|
+------+------+---------+---------

##12.Get shooterid, situation, minute, shotResult with rollup

In [ ]:
sql_str = "SELECT shooterId, situation, minute , shotResult FROM shots"\
+" group by shooterId, situation, shotResult,minute WITH ROLLUP ORDER BY shooterId"
shots_roll_up = spark.sql( sql_str )
shots_roll_up.show()

+---------+----------+------+-----------+
|shooterId| situation|minute| shotResult|
+---------+----------+------+-----------+
|     null|      null|  null|       null|
|        2|FromCorner|  null|       null|
|        2|  SetPiece|    67|    OwnGoal|
|        2|  SetPiece|  null|       null|
|        2|  SetPiece|  null|  SavedShot|
|        2|  SetPiece|  null|    OwnGoal|
|        2|FromCorner|    93|MissedShots|
|        2|      null|  null|       null|
|        2|FromCorner|  null|MissedShots|
|        2|  SetPiece|     1|  SavedShot|
|        2|FromCorner|    58|MissedShots|
|        3|FromCorner|  null|  SavedShot|
|        3|  OpenPlay|  null|       Goal|
|        3|FromCorner|    53|MissedShots|
|        3|  SetPiece|    57|MissedShots|
|        3|  OpenPlay|    21|  SavedShot|
|        3|FromCorner|    15|MissedShots|
|        3|FromCorner|    89|BlockedShot|
|        3|  OpenPlay|  null|MissedShots|
|        3|FromCorner|  null| ShotOnPost|
+---------+----------+------+-----

##13.Playerid, his name, total own goals in different season

In [ ]:
sql_str = "SELECT a.playerId, p.name, SUM(ownGoals) as ownGoals, g.season  FROM appearances a JOIN players p ON"\
+" (a.playerId = p.PlayerId) JOIN games g ON(a.gameId = g.gameId) GROUP BY a.playerId, p.name, g.season WITH ROLLUP"\
+" ORDER BY a.playerId, g.season"

shots_roll_up = spark.sql( sql_str )
shots_roll_up.show()

+--------+------------------+--------+------+
|playerId|              name|ownGoals|season|
+--------+------------------+--------+------+
|    null|              null|    1014|  null|
|       1|Christian Mathenia|       0|  null|
|       1|              null|       0|  null|
|       1|Christian Mathenia|       0|  2015|
|       1|Christian Mathenia|       0|  2016|
|       1|Christian Mathenia|       0|  2017|
|       1|Christian Mathenia|       0|  2018|
|       2|              null|       1|  null|
|       2|     Gy�rgy Garics|       1|  null|
|       2|     Gy�rgy Garics|       1|  2015|
|       3|    Luca Caldirola|       0|  null|
|       3|              null|       0|  null|
|       3|    Luca Caldirola|       0|  2014|
|       3|    Luca Caldirola|       0|  2015|
|       3|    Luca Caldirola|       0|  2016|
|       3|    Luca Caldirola|       0|  2017|
|       3|    Luca Caldirola|       0|  2020|
|       4|        Aytac Sulu|       3|  null|
|       4|              null|     

# Q09 PIVOT

##14.Get game id, home goals from season 2015,2016,2017,2018

In [ ]:

sql_str = "SELECT * FROM "\
  +"(SELECT gameId ,season as season, homeGoals FROM games  )  " \
  + "PIVOT ( sum(homeGoals) FOR season IN ( '2015','2016','2017','2018' ))"
pivoted = spark.sql( sql_str )
pivoted.show()

+------+----+----+----+----+
|gameId|2015|2016|2017|2018|
+------+----+----+----+----+
|  7340|null|null|   0|null|
| 11858|null|null|null|null|
|  2866|null|   1|null|null|
|  3997|null|   0|null|null|
| 15790|null|null|null|null|
| 14832|null|null|null|null|
|  7253|null|null|   1|null|
| 12799|null|null|null|null|
| 13285|null|null|null|null|
|  1088|   2|null|null|null|
|  3175|null|   3|null|null|
|  7554|null|null|   4|null|
| 10206|null|null|null|   0|
|   471|null|   2|null|null|
| 11748|null|null|null|null|
|   148|   2|null|null|null|
|  9376|null|null|null|   3|
|  3794|null|   1|null|null|
|   496|null|   1|null|null|
|  1580|   1|null|null|null|
+------+----+----+----+----+
only showing top 20 rows



##15.Get shooter id, shot result, total Excepted goals from 75, 34, 90, 13

In [ ]:


sql_str = "SELECT * FROM "\
  +"(SELECT shooterId, xGoal, minute , shotResult FROM shots ) " \
  + "PIVOT ( SUM(xGoal) FOR minute IN ( 75,34,90,13 ))"
pivoted = spark.sql( sql_str )
pivoted.show()

+---------+-----------+------------------+------------------+------------------+-------------------+
|shooterId| shotResult|                75|                34|                90|                 13|
+---------+-----------+------------------+------------------+------------------+-------------------+
|     2370|       Goal|0.4882115051150328|              null|0.5338043384253979| 0.5510657429695132|
|     2233|BlockedShot|              null|              null|              null|               null|
|     3795|MissedShots|0.0822890996932983|              null|1.2884900569915778| 0.3453409960493449|
|     3422|  SavedShot|              null| 0.105036497116089|  0.05373315513134|               null|
|      301|BlockedShot| 0.107572317123413|0.0515943244099617|              null|               null|
|      300|  SavedShot|0.0297455172985792|0.0677571892738342|              null| 0.0501434840261936|
|     4297|MissedShots|              null|              null|              null|  0.2469457